Initially, we import alll the necessary libraries.

In [503]:
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.preprocessing import StandardScaler
from random import shuffle

We import the necessary files and process them.

In [504]:
df_train = pd.read_csv('train_with_label.txt',sep='\t',header=None, error_bad_lines= False)
df_test = pd.read_csv('dev_with_label.txt', sep='\t',header=None, error_bad_lines= False)
df_test_nolabel =pd.read_csv('test_without_label.txt', sep='\t',header=None, error_bad_lines= False)

b'Skipping line 323: expected 4 fields, saw 5\nSkipping line 823: expected 4 fields, saw 5\nSkipping line 909: expected 4 fields, saw 5\nSkipping line 1013: expected 4 fields, saw 5\nSkipping line 1084: expected 4 fields, saw 5\nSkipping line 1097: expected 4 fields, saw 5\nSkipping line 1141: expected 4 fields, saw 5\nSkipping line 1267: expected 4 fields, saw 5\nSkipping line 1418: expected 4 fields, saw 5\nSkipping line 1486: expected 4 fields, saw 5\nSkipping line 1557: expected 4 fields, saw 5\nSkipping line 1675: expected 4 fields, saw 5\nSkipping line 1747: expected 4 fields, saw 5\nSkipping line 1801: expected 4 fields, saw 5\nSkipping line 1870: expected 4 fields, saw 5\nSkipping line 2025: expected 4 fields, saw 5\nSkipping line 2102: expected 4 fields, saw 5\nSkipping line 2371: expected 4 fields, saw 5\nSkipping line 2799: expected 4 fields, saw 5\nSkipping line 3016: expected 4 fields, saw 5\nSkipping line 3104: expected 4 fields, saw 5\nSkipping line 3317: expected 4 fiel

In [505]:
#setting column names for test and training set
df_train.columns = ["instance_id","sent1","sent2","gold_score"]
df_test.columns = ["instance_id","sent1","sent2","gold_score"]
df_test_nolabel.columns = ["instance_id","sent1","sent2"]

In [506]:
#remove all rows with null values
df_train = df_train.dropna()
print(df_train.isnull().sum())
df_test = df_test.dropna()
print(df_test.isnull().sum())
df_test_nolabel = df_test_nolabel.dropna()
print(df_test_nolabel.isnull().sum())
print(df_train.shape)
print(df_test.shape)
print(df_test_nolabel.shape)

instance_id    0
sent1          0
sent2          0
gold_score     0
dtype: int64
instance_id    0
sent1          0
sent2          0
gold_score     0
dtype: int64
instance_id    0
sent1          0
sent2          0
dtype: int64
(3850, 4)
(699, 4)
(946, 3)


In [507]:
#possible target names 
print("Train data target names: {}".format(df_train["gold_score"].unique()))
print("Test data target names: {}".format(df_test["gold_score"].unique()))

Train data target names: [1. 0.]
Test data target names: [0. 1.]


In [508]:
#Training dataset
df_train['sent1']=df_train['sent1'].astype('string')
df_train['sent2']=df_train['sent2'].astype('string')
df_train['sent1'] = df_train['sent1'].str.lower()
df_train['sent2'] = df_train['sent2'].str.lower()
#Test dataset
df_test['sent1']=df_test['sent1'].astype('string')
df_test['sent2']=df_test['sent2'].astype('string')
df_test['sent1'] = df_test['sent1'].str.lower()
df_test['sent2'] = df_test['sent2'].str.lower()
#Test with no label
df_test_nolabel['sent1']=df_test_nolabel['sent1'].astype('string')
df_test_nolabel['sent2']=df_test_nolabel['sent2'].astype('string')
df_test_nolabel['sent1'] = df_test_nolabel['sent1'].str.lower()
df_test_nolabel['sent2'] = df_test_nolabel['sent2'].str.lower()

In [509]:
print(df_train.dtypes)
print(df_test.dtypes)

instance_id     object
sent1           string
sent2           string
gold_score     float64
dtype: object
instance_id     object
sent1           string
sent2           string
gold_score     float64
dtype: object


In [510]:
#Training
df_train['sent1'] = df_train['sent1'].str.replace(",","")
df_train['sent2'] = df_train['sent2'].str.replace(",","")
#Testing
df_test['sent1'] = df_test['sent1'].str.replace(",","")
df_test['sent2'] = df_test['sent2'].str.replace(",","")
#Testing with no label
df_test_nolabel['sent1'] = df_test_nolabel['sent1'].str.replace(",","")
df_test_nolabel['sent2'] = df_test_nolabel['sent2'].str.replace(",","")

## Feature 1: WordCount difference

In [511]:
#Training
df_train['sent1l'] = df_train['sent1'].str.split()
df_train['sent2l'] = df_train['sent2'].str.split()
#Testing
df_test['sent1l'] = df_test['sent1'].str.split()
df_test['sent2l'] = df_test['sent2'].str.split()
#Testing with no label
df_test_nolabel['sent1l'] = df_test_nolabel['sent1'].str.split()
df_test_nolabel['sent2l'] = df_test_nolabel['sent2'].str.split()

In [512]:
#Training
df_train['sent1l']=df_train['sent1l'].apply(lambda x: len(x))
df_train['sent2l']=df_train['sent2l'].apply(lambda x: len(x))
#Testing
df_test['sent1l']=df_test['sent1l'].apply(lambda x: len(x))
df_test['sent2l']=df_test['sent2l'].apply(lambda x: len(x))
#Testing with no label
df_test_nolabel['sent1l']=df_test_nolabel['sent1l'].apply(lambda x: len(x))
df_test_nolabel['sent2l']=df_test_nolabel['sent2l'].apply(lambda x: len(x))

In [513]:
df_train = df_train.assign(wcd=lambda x: abs((x['sent1l']-x['sent2l'])))
df_test = df_test.assign(wcd=lambda x: abs((x['sent1l']-x['sent2l'])))
df_test_nolabel = df_test_nolabel.assign(wcd=lambda x: abs((x['sent1l']-x['sent2l'])))

In [514]:
print(df_test_nolabel.isnull().sum())

instance_id    0
sent1          0
sent2          0
sent1l         0
sent2l         0
wcd            0
dtype: int64


## Feature 2: Fuzzywuzzy ratio

In [515]:
from fuzzywuzzy import fuzz
import string

In [516]:
df_train['fuzz_ratio'] = df_train.apply(lambda row: fuzz.ratio(row['sent1'].translate(str.maketrans('', '', string.punctuation)), row['sent2'].translate(str.maketrans('', '', string.punctuation))), axis = 1)
df_test['fuzz_ratio'] = df_test.apply(lambda row: fuzz.ratio(row['sent1'].translate(str.maketrans('', '', string.punctuation)), row['sent2'].translate(str.maketrans('', '', string.punctuation))), axis = 1)
df_test_nolabel['fuzz_ratio'] = df_test_nolabel.apply(lambda row: fuzz.ratio(row['sent1'].translate(str.maketrans('', '', string.punctuation)), row['sent2'].translate(str.maketrans('', '', string.punctuation))), axis = 1)


In [517]:
df_train['fuzz_token_sort_ratio'] = df_train.apply(lambda row: fuzz.token_sort_ratio(row['sent1'].translate(str.maketrans('', '', string.punctuation)), row['sent2'].translate(str.maketrans('', '', string.punctuation))), axis = 1)
df_test['fuzz_token_sort_ratio'] = df_test.apply(lambda row: fuzz.token_sort_ratio(row['sent1'].translate(str.maketrans('', '', string.punctuation)), row['sent2'].translate(str.maketrans('', '', string.punctuation))), axis = 1)
df_test_nolabel['fuzz_token_sort_ratio'] = df_test_nolabel.apply(lambda row: fuzz.token_sort_ratio(row['sent1'].translate(str.maketrans('', '', string.punctuation)), row['sent2'].translate(str.maketrans('', '', string.punctuation))), axis = 1)

In [518]:
# df_train

## Feature 3: Bleu score

In [519]:
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.bleu_score import SmoothingFunction

In [520]:
df_train['bleu_score'] = df_train.apply(lambda row: sentence_bleu(row['sent1'].translate(str.maketrans('', '', string.punctuation)), row['sent2'].translate(str.maketrans('', '', string.punctuation)),smoothing_function=SmoothingFunction().method4), axis = 1)
df_test['bleu_score'] = df_test.apply(lambda row: sentence_bleu(row['sent1'].translate(str.maketrans('', '', string.punctuation)), row['sent2'].translate(str.maketrans('', '', string.punctuation)),smoothing_function=SmoothingFunction().method4),axis = 1)
df_test_nolabel['bleu_score'] = df_test_nolabel.apply(lambda row: sentence_bleu(row['sent1'].translate(str.maketrans('', '', string.punctuation)), row['sent2'].translate(str.maketrans('', '', string.punctuation)),smoothing_function=SmoothingFunction().method4),axis = 1)

In [521]:
# df_test_nolabel

## Reshaping

In [522]:
print(df_train.dtypes)
print(df_test.dtypes)
print(df_test_nolabel.dtypes)

instance_id               object
sent1                     string
sent2                     string
gold_score               float64
sent1l                     int64
sent2l                     int64
wcd                        int64
fuzz_ratio                 int64
fuzz_token_sort_ratio      int64
bleu_score               float64
dtype: object
instance_id               object
sent1                     string
sent2                     string
gold_score               float64
sent1l                     int64
sent2l                     int64
wcd                        int64
fuzz_ratio                 int64
fuzz_token_sort_ratio      int64
bleu_score               float64
dtype: object
instance_id               object
sent1                     string
sent2                     string
sent1l                     int64
sent2l                     int64
wcd                        int64
fuzz_ratio                 int64
fuzz_token_sort_ratio      int64
bleu_score               float64
dtype: object


In [523]:
X_train = df_train.drop(columns=['instance_id','sent1','sent2','gold_score','sent1l','sent2l']).values
X_test = df_test.drop(columns=['instance_id','sent1','sent2','gold_score','sent1l','sent2l']).values
y_train = df_train['gold_score'].values
y_test =df_test['gold_score'].values
X_test_nolabel = df_test_nolabel.drop(columns=['instance_id','sent1','sent2','sent1l','sent2l']).values

normalizer = StandardScaler()
# X_train_val = X_train_val.reshape(-1,1)
# X_test = X_test.reshape(-1,1)
X_train = normalizer.fit_transform(X_train)
X_test = normalizer.transform(X_test)
X_test_nolabel = normalizer.transform(X_test_nolabel)
print("train_val: {}, test: {}, test_nolabel: {}".format(X_train.shape, X_test.shape, X_test_nolabel.shape))


train_val: (3850, 4), test: (699, 4), test_nolabel: (946, 4)


## Logistic Regression model

In [524]:
clflr = LogisticRegression(penalty='l2', solver='lbfgs')
clflr.fit(X_train_val, y_train_val)

# evaluate the model on the testing set
y_test_pred = clflr.predict(X_test)


acc = accuracy_score(y_test, y_test_pred)
f1 = f1_score(y_test, y_test_pred)
recall = recall_score(y_test, y_test_pred)
precision = precision_score(y_test, y_test_pred)


print("accuracy: {:.3f}, recall: {:.3f}, precision: {:.3f}, f1: {:.3f},".format(acc, recall, precision, f1))

accuracy: 0.519, recall: 0.994, precision: 0.511, f1: 0.675,


## SVM Model

In [525]:
clf = svm.SVC()
clf.fit(X_train, y_train)

# evaluate the model on the testing set
y_test_pred = clf.predict(X_test)


acc = accuracy_score(y_test, y_test_pred)
f1 = f1_score(y_test, y_test_pred)
recall = recall_score(y_test, y_test_pred)
precision = precision_score(y_test, y_test_pred)


print("accuracy: {:.3f}, recall: {:.3f}, precision: {:.3f}, f1: {:.3f},".format(acc, recall, precision, f1))

accuracy: 0.612, recall: 0.952, precision: 0.568, f1: 0.711,


## Prediction for test dataset

In [526]:
y_test_pred = clf.predict(X_test_nolabel)

In [527]:
iterations = df_test_nolabel['instance_id'].to_numpy()
# print(dt)
print(len(iterations))

946


In [528]:
# print(y_test_pred)
print(len(y_test_pred))

946


In [529]:
file = open('AbuHasnatHasib_test_result.txt', 'w') #write to file

for i in range(0,len(iterations)-1):
    file.write(str(iterations[i]) + "\t" + str(y_test_pred[i]) + "\n")


In [530]:
file.close() #close file